In [ ]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 499 (delta 208), reused 200 (delta 200), pack-reused 274 (from 1)
Receiving objects: 100% (499/499), 3.05 MiB | 21.72 MiB/s, done.
Resolving deltas: 100% (308/308), done.


In [ ]:
%cd deep-text-recognition-benchmark/

/content/deep-text-recognition-benchmark


In [ ]:
!pip install -q fire
!pip install -q lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
import lmdb

In [ ]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From (original): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
From (redirected): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9&confirm=t&uuid=8e65627a-68ef-4cb5-8358-43d9e6bb999c
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:02<00:00, 68.3MB/s]


In [ ]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder my_test/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
my_test/Text_fa.png      	reconsing                	0.0000
my_test/persian_plate.png	tagysher                 	0.0002
my_test/plate.png        	284fh3                   	0.5594
my_test/

In [ ]:
!gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM

Downloading...
From (original): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
From (redirected): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM&confirm=t&uuid=eda6be75-3ad3-41e5-b35f-1c36a6d528f4
To: /content/deep-text-recognition-benchmark/plate_img-train.zip
100% 196M/196M [00:03<00:00, 61.1MB/s]


In [ ]:
!gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I

Downloading...
From (original): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
From (redirected): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I&confirm=t&uuid=5cf69cfb-026c-46c7-b3cf-5114cf2b404d
To: /content/deep-text-recognition-benchmark/plate_img-validation.zip
100% 27.2M/27.2M [00:00<00:00, 108MB/s]


In [ ]:
from zipfile import ZipFile

zip_path = "/content/deep-text-recognition-benchmark/plate_img-train.zip"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_content = zip_ref.namelist()

for name in zip_content[:30]:  # فقط ۳۰ تا اولی برای نمونه
    print(name)

train/
train/00001.jpg
train/00001.xml
train/00002.jpg
train/00002.xml
train/00004.jpg
train/00004.xml
train/00006.jpg
train/00006.xml
train/00008.jpg
train/00008.xml
train/00011.jpg
train/00011.xml
train/00014.jpg
train/00014.xml
train/00015.jpg
train/00015.xml
train/00016.jpg
train/00016.xml
train/00017.jpg
train/00017.xml
train/00021.jpg
train/00021.xml
train/00022.jpg
train/00022.xml
train/00023.jpg
train/00023.xml
train/00024.jpg
train/00024.xml
train/00025.jpg


In [ ]:
import os
import xml.etree.ElementTree as ET
from zipfile import ZipFile

zip_path = "/content/deep-text-recognition-benchmark/plate_img-train.zip"
extract_dir = "/content/plate_data"
image_dir = os.path.join(extract_dir, "train")  # درست شد

# 1. استخراج فایل zip
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 2. آماده‌سازی فایل خروجی
output_file = os.path.join(extract_dir, "labels.txt")

with open(output_file, "w", encoding="utf-8") as out:
    for fname in os.listdir(image_dir):
        if fname.endswith(".xml"):
            xml_path = os.path.join(image_dir, fname)
            tree = ET.parse(xml_path)
            root = tree.getroot()

            chars = []
            for obj in root.findall("object"):
                char = obj.find("name").text
                chars.append(char)

            label = "".join(chars)
            img_name = fname.replace(".xml", ".jpg")
            img_path = f"train/{img_name}"  # درست شد
            out.write(f"{img_path}\t{label}\n")

print("✅ labels.txt ساخته شد.")


In [ ]:
!python create_lmdb_dataset.py \
  --inputPath path_to_images \
  --gtFile path_to_labels.txt \
  --outputPath path_to_lmdb_dataset

SyntaxError: invalid syntax (<ipython-input-19-737d8d2e4340>, line 1)